### Task 1: Age estimation regression based on features

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import collections 
import datetime 
from datetime import date
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn import preprocessing, datasets
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

from sklearn.metrics import r2_score

from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, RFECV
from sklearn.svm import SVR

import tensorflow as tf
from sklearn.ensemble import IsolationForest, AdaBoostRegressor, ExtraTreesRegressor
import tensorflow as tf
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer


import sklearn
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import RobustScaler
from sklearn.impute import KNNImputer

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

from sklearn.datasets import load_digits
from sklearn.manifold import LocallyLinearEmbedding

from sklearn import linear_model

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.inspection import permutation_importance

#import missingno as msno  
from playsound import playsound
from IPython.display import Audio
import pygame

def notification():
    sound_file = 'sound.mp3'
    pygame.mixer.init()
    pygame.mixer.music.load(sound_file)
    pygame.mixer.music.play()

    '''
Plot training deviance
''' 
def plotDeviance():
    test_score = np.zeros((params["n_estimators"],), dtype=np.float64)
    for i, pred in enumerate(regr.staged_predict(X_test)):
        test_score[i] = regr.loss_(pred,y_test)

    fig = plt.figure(figsize=(6, 6))
    plt.subplot(1, 1, 1)
    plt.title("Deviance")
    plt.plot(
        np.arange(params["n_estimators"]) + 1,
        regr.train_score_,
        "b-",
        label="Training Set Deviance",
    )
    plt.plot(
        np.arange(params["n_estimators"]) + 1, test_score, "r-", label="Test Set Deviance"
    )
    plt.legend(loc="upper right")
    plt.xlabel("Boosting Iterations")
    plt.ylabel("Deviance")
    fig.tight_layout()
    plt.show()

    print(test_score.mean())
    mse = mean_squared_error(y_test, regr.predict(X_test))
    print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

def scaler(X_train):
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    return X_train

### Load data

In [10]:
X_train = pd.read_csv('X_train.csv').drop(columns = ['id'])
y_train = pd.read_csv('y_train.csv').drop(columns = ['id'])
X_test_h = pd.read_csv('X_test.csv').drop(columns = ['id'])
#X_train = X_train.fillna(X_train.mean())
#msno.matrix(X_train)
#msno.heatmap(X_train)

#X_train = X_train.fillna(X_train.mean())

#X_train = X_train[relevant_features]


#print(min(X_train.var()), max(X_train.var()))
#print(X_train.var()[X_train.var()>8000000])

### Filling missing values

In [11]:
'''
Filling missing Values
'''

#imp = KNNImputer(n_neighbors=200, weights="distance")
#imp = MissForest()
#imp = IterativeImputer(max_iter=10, random_state=1, n_nearest_features = 100, verbose = 2)
#imp = BiScaler()
#X_train = imp.fit_transform(np.array(X_train))
imp = IterativeImputer(max_iter=10, random_state=1, n_nearest_features = 20, verbose = 0)
X_train = imp.fit_transform(np.array(X_train))
print('Imputation Done')
#X_train = pd.DataFrame(imp.transform(np.array(X_train)))

Imputation Done


### Outlier detection and removal

In [12]:
'''
clustering = DBSCAN(eps=0.001, min_samples=3).fit(X_train)
mask = clustering.labels_ != -1
X_train, y_train =  np.array(X_train)[mask, :], np.array(y_train)[mask]
'''

clf = IsolationForest()
outliers = clf.fit_predict(X_train)
mask = outliers != -1
X_train, y_train =  np.array(X_train)[mask, :], np.array(y_train)[mask]


print('Outliers Removed, resulting shape:', X_train.shape, y_train.shape)


Outliers Removed, resulting shape: (1208, 832) (1208, 1)


### Feature selection

In [13]:
#scaler = MinMaxScaler(feature_range=(0, 1))
#X_train = scaler.fit_transform(X_train)
#

#X_train = scaler(X_train)

#
#scaler = Normalizer().fit(X_train)
#X_train = scaler.transform(X_train)


# PCA: unsupervised (se queda con 828 features)
    # print('shape before PCA is ', X_train.shape)
    # selector = PCA(n_components='mle', whiten=False, svd_solver='full')
    #selector.fit(X_train)  
    #X_train = selector.transform(X_train)
    #scalerPost = StandardScaler().fit(X_train)
    #X_train = scalerPost.inverse_transform(X_train)
    #print (X_train.shape)


# Create and fit selector (univariate feature selection)
k0=80
selector = SelectKBest(mutual_info_regression, k=k0)
selector.fit(X_train,  np.ravel(y_train))
# Get columns to keep and create new dataframe with those only
cols = selector.get_support(indices=True)
X_train = pd.DataFrame(X_train)
X_test_h = pd.DataFrame(X_test_h)
#
#
X_train = X_train.iloc[:,cols]
X_test_h = X_test_h.iloc[:,cols]


#X_train = SelectKBest(f_regression, k=k0).fit_transform(X_train, np.ravel(y_train))



print('Feature Selection Done')
print('Shape after feature selection:', X_train.shape)
notification()

Feature Selection Done
Shape after feature selection: (1208, 80)


In [ ]:
#cols2 = selector.ranking_==1
cols2 = np.loadtxt('ranking200.txt') == 1
X_train = pd.DataFrame(X_train)
X_test_h = pd.DataFrame(X_test_h)
#
#
X_train = X_train.iloc[:,cols2]
X_test_h = X_test_h.iloc[:,cols2]
#X_train = RFECV(f_regression, k=200).fit_transform(X_train, np.ravel(y_train))

print(X_train.shape)

Outliers Removed, resulting shape: (1159, 80) (1159, 1)


### Normalization, Data split, data augmentation

In [14]:
'''
Normalization
'''
#scaler = MinMaxScaler(feature_range=(0, 1))
#X_train = scaler.fit_transform(X_train)
#
#scaler = StandardScaler().fit(X_train)
#X_train = scaler.transform(X_train)
#
#scaler = Normalizer().fit(X_train)
#X_train = scaler.transform(X_train)
#X_train = scaler(X_train)



'''
Data Split
'''

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1, shuffle = True)
X_train = np.array(X_train)
#X_test = np.array(X_test)
y_train = np.array(y_train)
#y_test = np.array(y_test)



'''
Data Augmentation
'''

#print(pd.DataFrame(X_train).columns)
#concat_data = pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train)], axis = 1)


#concat = np.concatenate((X_train, y_train), axis = 1)
#concat = pd.DataFrame(concat)
#concat.columns = concat.columns.astype(str)

#concat = concat.rename(columns={concat.columns[-1]: "y"})

#import smogn

#housing_smogn = smogn.smoter(data = concat,y = concat.columns[-1])

#X_train = pd.DataFrame(housing_smogn[housing_smogn.columns[:-1]])
#y_train = pd.DataFrame(housing_smogn[housing_smogn.columns[-1]])

'\nData Augmentation\n'

### Training

In [258]:
'''
Training
'''


paramsH = {#"random_state": 1, 
"learning_rate": 0.1, 
"l2_regularization": 1.5,
#"max_iter": 1000,
"max_depth": 3,
#"scoring": 'f1_micro',
}

paramsRF_random = {'n_estimators': 1000,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 43,
 'bootstrap': True}

params = {"random_state": 1, 
"learning_rate": 0.1, 
"n_estimators": 3000, 
"max_depth": 5,
"max_features":'sqrt',
"warm_start":True
}


#modelfit(xgb1, train, predictors)

#regr = GradientBoostingRegressor(random_state=1, loss = 'squared_error', learning_rate = 0.1, n_estimators=1000, max_depth= 3)
regr = GradientBoostingRegressor(**params)
#regr = HistGradientBoostingRegressor(**paramsH)

#regr = AdaBoostRegressor(base_estimator=None,random_state=1, loss = 'square', learning_rate = 0.1, n_estimators=1000)
#regr = RandomForestRegressor(n_estimators = 1000, verbose=0, n_jobs=-1)
#regr = RandomForestRegressor(**paramsRF_random)

#regr = AdaBoostClassifier(n_estimators=1000)
#regr = MLPRegressor(hidden_layer_sizes=([512, 256, 128, 64]), random_state=1, max_iter=2000, verbose = False, learning_rate_init=0.001, early_stopping=True)
scores = cross_val_score(regr, X_train, np.ravel(y_train), cv=5, scoring='r2')
print('Total CV Scores:',scores)
print("CV Score : Mean - %.7g | Std - %.7g | Max - %.7g | Min - %.7g" % (np.mean(scores),np.std(scores),np.max(scores),np.min(scores)))
        
'''
Performance
'''

#reg = make_pipeline(StandardScaler(with_mean=False), LinearRegression())
regr.fit(X_train, np.ravel(y_train))
#pred = reg.predict(X_test)
#X_test = xgb.DMatrix(X_test)
#pred = regr.predict(X_test)
#print(min(pred), max(pred))
#pygame.mixer.init()
#pygame.mixer.music.load(sound_file)
#pygame.mixer.music.play()
#print("FINAL SCORE", r2_score(y_test, pred))

notification()

#plotDeviance()


Total CV Scores: [0.71370931 0.63070532 0.60032783 0.59324576 0.65819184]
CV Score : Mean - 0.639236 | Std - 0.0438345 | Max - 0.7137093 | Min - 0.5932458


### Training 2.0: without data split (XGBoost Regression)

In [16]:
'''
Training without data split
'''
regr = XGBRegressor(learning_rate =0.1,
    n_estimators=5000,
    max_depth=9,
    min_child_weight=3,
    min_split_loss=0.2,
    subsample=0.9,
    sampling_method = 'uniform', #uniform + subsample >= 0.5
    colsample_bytree=0.7, #=subsample,
    objective= 'reg:squarederror',
    #nthread=4,
    #scale_pos_weight=1,
    seed=0,
    tree_method = 'auto',
    eval_metric = 'rmse',
    reg_alpha='0.1'
    )
regr = XGBRegressor(learning_rate =0.1,
    n_estimators=3000,
    max_depth=9,
    min_child_weight=3,
    min_split_loss=0.2,
    subsample=0.9,
    sampling_method = 'uniform', #uniform + subsample >= 0.5
    colsample_bytree=0.7, #=subsample,
    objective= 'reg:squarederror',
    #nthread=4,
    #scale_pos_weight=1,
    seed=0,
    tree_method = 'auto',
    eval_metric = 'rmse',
    reg_alpha='0.1'
    )
#modelfit(xgb1, train, predictors)

#regr = GradientBoostingRegressor(random_state=1, loss = 'squared_error', learning_rate = 0.1, n_estimators=1000, max_depth= 3)
#regr = GradientBoostingRegressor(**params)
#regr = HistGradientBoostingRegressor(**paramsH)

#regr = AdaBoostRegressor(base_estimator=None,random_state=1, loss = 'square', learning_rate = 0.1, n_estimators=1000)
#regr = RandomForestRegressor(n_estimators = 1000, verbose=0, n_jobs=-1)
#regr = RandomForestRegressor(**paramsRF_random)

#regr = AdaBoostClassifier(n_estimators=1000)
#regr = MLPRegressor(hidden_layer_sizes=([512, 256, 128, 64]), random_state=1, max_iter=2000, verbose = False, learning_rate_init=0.001, early_stopping=True)
scores = cross_val_score(regr, X_train, np.ravel(y_train), cv=5, scoring='r2')
print('Total CV Scores:',scores)
print("CV Score : Mean - %.7g | Std - %.7g | Max - %.7g | Min - %.7g" % (np.mean(scores),np.std(scores),np.max(scores),np.min(scores)))
        
'''
Performance
'''

#reg = make_pipeline(StandardScaler(with_mean=False), LinearRegression())
regr.fit(X_train, np.ravel(y_train))
pred = regr.predict(np.float32(X_test))
#pred = regr.predict(X_test)
#print(min(pred), max(pred))
#pygame.mixer.init()
#pygame.mixer.music.load(sound_file)
#pygame.mixer.music.play()
print("FINAL SCORE", r2_score(y_test, pred))

notification()

Total CV Scores: [0.42896075 0.52009719 0.56036437 0.58020332 0.61474382]
CV Score : Mean - 0.5408739 | Std - 0.06378036 | Max - 0.6147438 | Min - 0.4289608
FINAL SCORE 0.5103752346626063


### Prediction in test

In [259]:
test_data = pd.read_csv('X_test.csv')
X_test_h = np.array(test_data.loc[:, test_data.columns != 'id'])

#X_test_h = scaler(X_test_h)
imp = IterativeImputer(max_iter=10, random_state=1, n_nearest_features = 20, verbose = 0)
X_test_h = imp.fit_transform(np.array(X_test_h))
X_test_h = pd.DataFrame(imp.transform(X_test_h))

X_test_h = X_test_h.iloc[:,cols]
#X_test_h = scaler(X_test_h)


pred_test = test_data.loc[:,['id']]

#X_test_h = xgb.DMatrix(X_test_h)
predictions = pred = regr.predict(np.float32(X_test_h))
#predictions = pred =  regr.predict(X_test_h)
pred_test['y'] = predictions

print(min(pred_test['y']), max(pred_test['y']))

50.89493102608006 86.48822668857551


In [260]:
pred_test.to_csv('outputXGB5-warmstateGB.csv',index=False)

In [ ]:
#pred_test.to_csv('results' + str(datetime.date.today().strftime("%m/%d/%Y, %H:%M:%S")) + '.csv',index=False)
pred_test.to_csv('output '+ str(datetime.datetime.now()) + '.csv',index=False)

### Extra: Hyperparameter tuning of Gradient Boosting

In [ ]:
'''
Gradient Boosting hyperparameter tuning
'''

# First: Random search with CV
from sklearn.model_selection import RandomizedSearchCV

# TREE PARAM
    # Minimum number of samples required at each leaf node (terminal)
        # it should be small for imbalanced classes
min_samples_leaf = [1, 2, 4]

# Minimum number of samples required to split a node: 
#   too low tends to overfit, too high tends to underfit
min_samples_split = [5, 8, 12]

# Number of features to consider at every split
max_features = ['auto', 'sqrt', 60] #up to 30,40% of total features

# Maximum number of levels in tree: 
#   higher depth, higher overfit (learn specific relations)
max_depth = [int(x) for x in np.linspace(10, 110, num = 4)]
max_depth.append(None)

# BOOSTING PARAM
# Method of selecting samples for training each tree
bootstrap = [True]

# Learning rate
#   smaller values will make the model robust

# Number of trees 
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 5000, num = 10)]

# Subsample: fraction of observations to be selected for each tree
#   if < 1.0 tends to a reduction of variance, increase in bias
subsample = 0.8
n_iter_no_change = 100
validation_fraction = 0.1

# MISCELLANEOUS PARAM
loss = 'squared_error' #default works well
random_state = None # we could define it
warm_start = False #if True it calls the previous call to fit and add more estimators to the ensemble

'''
random_grid = {'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
'bootstrap': bootstrap}

'''


In [ ]:
'''
XGB Boosting hyperparameter tuning
'''

regr = XGBRegressor(learning_rate =0.05,
    n_estimators=2000,
    max_depth=9,
    min_child_weight=3,
    min_split_loss=0.2,
    subsample=0.59,
    sampling_method = 'uniform', #uniform + subsample >= 0.5
    colsample_bytree=0.7, #=subsample,
    objective= 'reg:squarederror',
    nthread=4,

    #scale_pos_weight=1,
    seed=0,
    tree_method = 'auto',
    eval_metric = 'rmse'
    )

param_test = {'reg_alpha':[1e-5, 1e-2,0, 0.1, 1, 100]
}

gsearch1 = GridSearchCV(estimator=regr, param_grid=param_test,
                              scoring='r2', 
                              cv = 10, verbose=2,  n_jobs=-1)

gsearch1.fit(X_train, np.ravel(y_train))


print(gsearch1.best_params_)
print(gsearch1.best_score_)
notification()

### Helper: Plot of performance

In [ ]:
from matplotlib.pylab import rcParams
from sklearn.model_selection import GridSearchCV
rcParams['figure.figsize'] = 12, 4

def modelfit(regr, dtrain, predictors, performCV=True, printFeatureImportance=True, cv_folds=10):
    #Fit the algorithm on the data
    regr.fit(X_train, np.ravel(y_train))
        
    #Predict training set:
    pred = regr.predict(X_test)
    #dtrain_predprob = regr.predict_proba(X_test)[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_val_score(regr,  X_train, np.ravel(y_train), cv=cv_folds, scoring='r2')

    #Print model report:
    print("\nModel Report")
    print("Accuracy R2 : %.4g" % (r2_score(y_test, pred)) )
    #print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
    
    if performCV:
        print("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        print('Total CV Scores:',scores)
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(regr.feature_importances_).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

gbm0 = GradientBoostingRegressor(random_state=10, n_estimators = 600, max_features='sqrt')
modelfit(gbm0, X_train, np.ravel(y_train))

print("First gridsearch")
# Define parameters
min_samples_leaf = 4
min_samples_split = 12
max_features = 'sqrt' #60
max_depth = 6
subsample = 0.8
learning_rate = 0.1
random_state = 10

#n_estimators = [int(x) for x in np.linspace(start = 20, stop = 2000, num = 10)]


params1 = {#'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
'subsample': subsample,
'learning_rate': learning_rate,
'random_state': random_state }

param_test1 = {'n_estimators':range(100,3500,500)}
gbm1 = GradientBoostingRegressor(**params1)


gbm1 = GradientBoostingRegressor(**params1)

gsearch1 = GridSearchCV(estimator=gbm1, param_grid=param_test1,
                              scoring='r2', 
                              cv = 10, verbose=2,  n_jobs=-1)
gsearch1.fit(X_train, np.ravel(y_train))


print(gsearch1.best_params_)
print(gsearch1.best_score_)



### Extra: random forest hyperparameter tuning

In [ ]:
'''
Random Forest hyperparameter
'''

# First: Random search with CV
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 3)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 60] #up to 30,40% of total features
# Maximum number of levels in tree: higher depth, higher overfit (learn specific relations)
max_depth = [int(x) for x in np.linspace(10, 110, num = 4)]
max_depth.append(None)

# Minimum number of samples required to split a node: 
#   too low tends to overfit, too high tends to underfit
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node (terminal)
    # it should be small for imbalanced classes
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
rf_random.fit(X_train, np.ravel(y_train))

rf_random.best_params_

### Extra: Grid search with Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {  'bootstrap': [True],
 'max_depth': [5, 10, None],
 'max_features': ['auto', 'log2'],
 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]
  
  }

 # Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train, np.ravel(y_train))
grid_search.best_params_

best_gridRF = grid_search.best_estimator_ #model

### Handing in the results

In [ ]:
test_data = pd.read_csv('X_test.csv')
X_test_h = np.array(test_data.loc[:, test_data.columns != 'id'])

imp = IterativeImputer(max_iter=10, random_state=1, n_nearest_features = 10, verbose = 0)
X_test_h = imp.fit_transform(np.array(X_test_h))
X_test_h = pd.DataFrame(imp.transform(X_test_h))

X_test_h = X_test_h.iloc[:,cols]




pred_test = test_data.loc[:,['id']]
predictions = pred = regr.predict(X_test_h)
pred_test['y'] = predictions
#